In [45]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [46]:
%pig_start

In [47]:
%timeout 300

In [48]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
rm: `output/*': No such file or directory
rm: `output2/*': No such file or directory
rmdir: `output': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [49]:
!hadoop fs -put *.tsv .
!hadoop fs -ls

put: `data.tsv': File exists
Found 3 items
-rw-r--r--   1 root supergroup        615 2020-01-18 23:22 data.csv
-rw-r--r--   1 root supergroup       1741 2020-01-18 22:51 data.tsv
-rw-r--r--   1 root supergroup    2271958 2020-01-18 22:32 truck_event_text_partition.csv


In [50]:
!rm -rf input
!rm -rf output
!mkdir input
!mkdir output

In [51]:
%%writefile script.pig

-- crea la carpeta input in el HDFS
-- fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put input/ .

-- carga de datos
--Punto 7
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:BAG{}, f3:MAP[]);
    
r = FOREACH u GENERATE $0, COUNT_STAR($1) as ld1:CHARARRAY, SIZE($2) as ld2:CHARARRAY;

rr = FOREACH r GENERATE CONCAT($0, ',' , ld1 , ',' , ld2) as final;

y = ORDER rr BY final;

DUMP y;

-- escribe el archivo de salida
STORE y INTO 'output';

-- copia los archivos del HDFS al sistema local
fs -get output/ .

Overwriting script.pig


In [52]:
!pig -execute 'run script.pig'

2020-01-19 04:17:41,492 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-01-19 04:17:45,396 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-01-19 04:17:45,723 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-01-19 04:17:45,730 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-01-19 04:17:45,758 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.replication
2020-01-19 04:17:46,501 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, us

In [53]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2020-01-19 04:20 output/_SUCCESS
-rw-r--r--   1 root supergroup        240 2020-01-19 04:20 output/part-r-00000


In [54]:
!hadoop fs -cat output/part-r-00000

A,2,4
A,2,5
A,2,5
A,2,5
A,3,3
A,3,4
A,3,5
A,3,6
B,2,3
B,2,5
B,2,5
B,3,5
B,3,6
B,4,4
B,4,6
C,4,3
C,4,3
C,4,4
C,4,5
C,4,6
D,2,3
D,2,3
D,2,4
D,2,6
D,3,3
D,4,5
E,2,3
E,2,3
E,2,3
E,2,3
E,2,4
E,2,6
E,3,3
E,3,3
E,3,5
E,3,5
E,3,6
E,3,6
E,4,3
E,4,6


In [55]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-r-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [56]:
!rm -rf input
!rm -rf output